In [1]:
import numpy as np
import pandas as pd


from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

#import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import os
import datetime
from math import sqrt

import math
from tensorflow.keras import backend
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, SimpleRNN

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D, Dropout, Flatten
from keras.layers import  regularizers

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, LSTM, Flatten, Input, Reshape
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Lambda, Flatten
import tensorflow.keras.backend as K
from sklearn.model_selection import TimeSeriesSplit

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.python.keras.preprocessing.sequence import TimeseriesGenerator

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


***import data***

In [2]:

#import data
data = pd.read_csv("../VIDEO TRAFFIC/FINAL comparison -benchmark models/data/nbc news G16B3-QP = 28-1.csv",
                  names=['decoded_frame','arrival_time', 'frame', 
                                         'frame_size', 'x', 'y', 'z'])

In [3]:
data.head()

,decoded_frame,arrival_time,frame,frame_size,x,y,z
0,16,533.33333,I,18856,41.258,46.265,47.285
1,13,433.33333,B,2096,38.477,45.308,46.649
2,14,466.66667,B,2904,37.856,45.246,46.194
3,15,500.00000,B,3200,38.559,45.951,47.168
4,20,666.66667,P,20688,39.948,44.184,45.631


In [4]:
data.dropna(0,inplace= True)

In [5]:
#drop decoded frame column
data.drop(('decoded_frame'), axis=1, inplace=True)

data.drop(('x'), axis=1, inplace=True)
data.drop(('y'), axis=1, inplace=True)
data.drop(('z'), axis=1, inplace=True)

In [6]:
# Label encoding of species column numerically
le = LabelEncoder()
le.fit(data['frame'])
print(list(le.classes_))
data['frame'] = le.transform(data['frame'])

['B', 'I', 'P']


In [7]:
data.head(5)

,arrival_time,frame,frame_size
0,533.33333,1,18856
1,433.33333,0,2096
2,466.66667,0,2904
3,500.00000,0,3200
4,666.66667,2,20688


In [8]:
# 1  1 1 1 0
# 1 1 1  0

-1



-1

In [9]:
data = pd.get_dummies(data, columns = ['frame'])

In [10]:
from collections import defaultdict

b_series = defaultdict(list)
p_series = defaultdict(list)
i = []

repeat_size = 16

for idx, row in data.iterrows():
    fs = row['frame_size']
    fn = idx % repeat_size
    if fn == 0:
        # i frame
        i.append(fs)
    elif fn % 4 == 0:
        # p frame
        num_p = fn // 4 
        p_series[num_p].append(fs)
    else:
        # b frame
        num_b = fn - fn // 4
        b_series[num_b].append(fs)


In [11]:
print(len(i[:-1]))

3094


In [12]:
print(len(b_series[11]))

3094


In [13]:
frames_df = pd.DataFrame()

frames_df['I'] = i[:-1]

to_collect =  len(i) - 1

for num, series in b_series.items():
    frames_df[f'B{num}'] = series[:to_collect]
    
for num, series in p_series.items():
    frames_df[f'P{num}'] = series[:to_collect]
    
frames_df.head()

,I,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,P1,P2,P3
0,18856.0,2096.0,2904.0,3200.0,4048.0,7432.0,6392.0,5776.0,7088.0,10448.0,5352.0,7400.0,6912.0,20688.0,24688.0,24792.0
1,29648.0,5096.0,5032.0,2840.0,4880.0,3600.0,4824.0,19560.0,26208.0,16616.0,19056.0,17304.0,14648.0,20312.0,28392.0,21240.0
2,22944.0,11656.0,13736.0,13960.0,13224.0,15456.0,13720.0,25728.0,37352.0,36928.0,33648.0,36664.0,32368.0,16696.0,48880.0,46944.0
3,47392.0,30280.0,30536.0,28520.0,21928.0,27672.0,26320.0,19496.0,18872.0,13392.0,12912.0,16072.0,5784.0,33920.0,31080.0,38168.0
4,40360.0,9680.0,9520.0,1704.0,3936.0,4512.0,6488.0,15800.0,24152.0,20120.0,15632.0,18448.0,16528.0,26360.0,24488.0,13888.0


In [14]:
s = pd.Series(data['frame_size'])


lag_values = range(1, 10)

for l in lag_values:
    s1 = s.autocorr(l)
    print(l, s1)





1 0.012943787888281636
2 0.055558072706772696
3 0.004735795497954229
4 0.5170276608131649
5 -0.0021099752868512594
6 0.039906361117118865
7 -0.011846009365660111
8 0.49694708033436696
9 -0.017417705278670384


In [15]:
frames_df.corr()

,I,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,P1,P2,P3
I,1.000000,0.052262,0.054633,0.056422,0.066159,0.060053,0.050499,0.058581,0.061180,0.067366,0.055362,0.056301,0.056745,0.257049,0.241754,0.249404
B1,0.052262,1.000000,0.905069,0.813079,0.785781,0.788088,0.715723,0.664645,0.716148,0.674499,0.634110,0.651034,0.655437,0.604397,0.560401,0.540115
B2,0.054633,0.905069,1.000000,0.897565,0.818955,0.826511,0.801648,0.721340,0.748469,0.707265,0.683452,0.689948,0.658078,0.654270,0.606234,0.578239
B3,0.056422,0.813079,0.897565,1.000000,0.827825,0.823355,0.787527,0.734364,0.721517,0.663691,0.653878,0.672530,0.620906,0.640989,0.584941,0.555932
B4,0.066159,0.785781,0.818955,0.827825,1.000000,0.916239,0.855724,0.780854,0.796884,0.696572,0.678101,0.707392,0.689577,0.696868,0.628430,0.569762
B5,0.060053,0.788088,0.826511,0.823355,0.916239,1.000000,0.903953,0.817660,0.824102,0.769021,0.715968,0.739782,0.703524,0.732317,0.665952,0.606197
B6,0.050499,0.715723,0.801648,0.787527,0.855724,0.903953,1.000000,0.838509,0.813458,0.764773,0.752213,0.724080,0.677856,0.715455,0.657746,0.590877
B7,0.058581,0.664645,0.721340,0.734364,0.780854,0.817660,0.838509,1.000000,0.892209,0.809422,0.779446,0.777901,0.698580,0.679989,0.719368,0.633138
B8,0.061180,0.716148,0.748469,0.721517,0.796884,0.824102,0.813458,0.892209,1.000000,0.904817,0.845727,0.847759,0.813719,0.688992,0.735716,0.683631
B9,0.067366,0.674499,0.707265,0.663691,0.696572,0.769021,0.764773,0.809422,0.904817,1.000000,0.859792,0.842953,0.802017,0.639133,0.716642,0.681100


In [16]:
data.head(20)

,arrival_time,frame_size,frame_0,frame_1,frame_2
0,533.33333,18856,0,1,0
1,433.33333,2096,1,0,0
2,466.66667,2904,1,0,0
3,500.00000,3200,1,0,0
4,666.66667,20688,0,0,1
5,566.66667,4048,1,0,0
6,600.00000,7432,1,0,0
7,633.33333,6392,1,0,0
8,800.00000,24688,0,0,1
9,700.00000,5776,1,0,0
